In [27]:
import numpy as np

min_loop_length = 1

In [28]:
def nussinov_table(seq, gamma, min_loop_length=0):
    N = len(seq)
    table = [[0 for _ in range(N)] for _ in range(N)]
    for diff in range(1, N):  # Start from 1 to include zero loop length
        for i in range(N - diff):
            j = i + diff
            if j - i - 1 < min_loop_length:  # Enforcing the minimum loop length
                continue
            opt = -1
            if pair_check((seq[i], seq[j])):
                opt = table[i+1][j-1] + 1
            opt = max(opt, table[i+1][j], table[i][j-1])
            for k in range(i + 1, j):
                opt = max(opt, table[i][k] + table[k+1][j])
            table[i][j] = opt
    return table

def traceback(seq, table):
    i, j = 0, len(seq) - 1
    stack = [(i, j)]
    structure = []
    while stack:
        i, j = stack.pop()
        if i < j:
            if table[i+1][j] == table[i][j]:
                stack.append((i+1, j))
            elif table[i][j-1] == table[i][j]:
                stack.append((i, j-1))
            elif pair_check((seq[i], seq[j])) and table[i+1][j-1] + 1 == table[i][j]:
                structure.append((i, j))
                stack.append((i+1, j-1))
            else:
                for k in range(i + 1, j):
                    if table[i][k] + table[k+1][j] == table[i][j]:
                        stack.append((i, k))
                        stack.append((k+1, j))
                        break
    return structure

def write_structure(sequence, structure):
    dot_bracket = ["." for _ in range(len(sequence))]
    for s in structure:
        dot_bracket[min(s)] = "("
        dot_bracket[max(s)] = ")"
    return "".join(dot_bracket)

def nussinov(sequence, min_loop_length=0):
    gamma = {('A', 'U'), ('U', 'A'), ('C', 'G'), ('G', 'C')}
    table = nussinov_table(sequence, gamma, min_loop_length)
    structure = traceback(sequence, table)
    return write_structure(sequence, structure), structure

# Example usage
sequence = "GCAUAGC"
structure, pairs = nussinov(sequence)
print(f"Dot-Bracket Notation: {structure}")
print(f"Paired Bases: {pairs}")

Dot-Bracket Notation: ((.()))
Paired Bases: [(0, 6), (1, 5), (3, 4)]


In [31]:

def test_nussinov():
    test_sequences = {
        "seq1": {
            "sequence": "CGCGAAUUAGCGCGCGAAUUAGCG",
            "expected": "(((((((((((())))))))))))"
        },
        "seq2": {
            "sequence": "GGCGUAAGGAUUACCUAUGCC",
            "expected": "(((((.(((....))))))))"
        },
        "seq3": {
            "sequence": "GGCCAGAUUGAGCCUGGGAGCUCUCUGGCC",
            "expected": "(((((((..((((......)))))))))))"
        },
        "seq4": {
            "sequence": "GGGAAAUCC",
            "expected": ".((.(.)))"
        }
    }

    for key, value in test_sequences.items():
        sequence = value["sequence"]
        expected = value["expected"]
        result, pairs = nussinov(sequence)
        print(f"{key} - Sequence: {sequence}")
        print(f"Result: {result}")
      
        print()

# Execute the test function
test_nussinov()


seq1 - Sequence: CGCGAAUUAGCGCGCGAAUUAGCG
Result: ()()..(()((())).)()()().

seq2 - Sequence: GGCGUAAGGAUUACCUAUGCC
Result: .(()()(((()())))()())

seq3 - Sequence: GGCCAGAUUGAGCCUGGGAGCUCUCUGGCC
Result: .(((((()(.)()))))((())(.(.))))

seq4 - Sequence: GGGAAAUCC
Result: .((..()))

